In [1]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_classic.prompts import ChatPromptTemplate
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
load_dotenv()

c:\Users\yasho\OneDrive\Desktop\Krish-langchain\project8-yt-chatbot\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
c:\Users\yasho\OneDrive\Desktop\Krish-langchain\project8-yt-chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## 1. Document Ingestion


In [2]:
url = "https://youtu.be/YSHSHJKNi_U?si=wOmr9A3LZdF1HEsO"
video_id = ""

for i in url.split("/"):
    video_id = i[:11]

print(video_id)

YSHSHJKNi_U


In [3]:
ytt_api = YouTubeTranscriptApi()
fetched_transcript = ytt_api.fetch(video_id, languages=['en', 'hi', 'mr'])
print(fetched_transcript[0].text)

अगर पिछले कुछ सालों में और आने वाले कुछ


In [4]:
# flatten the transcript into a single string
transcript_text = " ".join([t.text for t in fetched_transcript])

In [5]:
transcript_text

'अगर पिछले कुछ सालों में और आने वाले कुछ सालों में ज्यादातर पैसा टेक जॉब्स में ही है तो क्या आपको ऐसा लगता है कि अगर आपके पास एक नॉन टेक डिग्री है तो आपकी जिंदगी में कुछ नहीं हो सकता तो यह वीडियो आप ही के लिए बनाया। कैसे आप एक नॉन टेकी होने के बावजूद एक टेक जॉब आज की दुनिया में लैंड कर सकते हैं जानने की कोशिश करेंगे। सबसे पहले एक 40 लाख का सवाल। फर्ज करिए कि आप जयपुर में रहने वाले एक 22 साल के कॉमर्स ग्रेजुएट हैं। मैं जनरलाइज़ कर रहा हूं लेकिन आपको मोस्ट लाइकली एक सेल्स एंड मार्केटिंग की नौकरी मिल जाएगी। 34 लाख साल की। 25 साल तक 44.5 लाख कमा रहे होंगे। 28 साल तक 5.5 6 लाख 30 साल तक 60 से ₹ लाख। मतलब अगले 5 सालों में आपकी तनख्वाह 26 से 28 लाख के बीच में होगी। वहीं अगर आप 2025 और 2026 में एक 6 महीने का बूट कैंप करते हैं। अ टेक बूट कैंप। 23 साल की उम्र होते-होते आपको एक डाटा एनालिस्ट की नौकरी मिल जाएगी जिसकी एवरेज सैलरी 6 से ₹7 लाख। 25 साल तक आप ₹12 लाख कमा रहे होंगे। 28 साल तक करीब 18 लाख और 30 साल तक करीब 23 से ₹25 लाख। 5 साल की आमदनी करीब 70 से 75 लाख। मतलब 5 साल के अंदर करीब 40 लाख का।

In [6]:
len(transcript_text)

23124

## 2. Splitting the Text into Chunks


In [7]:
chunks = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200).create_documents([transcript_text])

In [8]:
print(chunks[0].page_content)

अगर पिछले कुछ सालों में और आने वाले कुछ सालों में ज्यादातर पैसा टेक जॉब्स में ही है तो क्या आपको ऐसा लगता है कि अगर आपके पास एक नॉन टेक डिग्री है तो आपकी जिंदगी में कुछ नहीं हो सकता तो यह वीडियो आप ही के लिए बनाया। कैसे आप एक नॉन टेकी होने के बावजूद एक टेक जॉब आज की दुनिया में लैंड कर सकते हैं जानने की कोशिश करेंगे। सबसे पहले एक 40 लाख का सवाल। फर्ज करिए कि आप जयपुर में रहने वाले एक 22 साल के कॉमर्स ग्रेजुएट हैं। मैं जनरलाइज़ कर रहा हूं लेकिन आपको मोस्ट लाइकली एक सेल्स एंड मार्केटिंग की नौकरी मिल जाएगी। 34 लाख साल की। 25 साल तक 44.5 लाख कमा रहे होंगे। 28 साल तक 5.5 6 लाख 30 साल तक 60 से ₹ लाख। मतलब अगले 5 सालों में आपकी तनख्वाह 26 से 28 लाख के बीच में होगी। वहीं अगर आप 2025 और 2026 में एक 6 महीने का बूट कैंप करते हैं। अ टेक बूट कैंप। 23 साल की उम्र होते-होते आपको एक डाटा एनालिस्ट की नौकरी मिल जाएगी जिसकी एवरेज सैलरी 6 से ₹7 लाख। 25 साल तक आप ₹12 लाख कमा रहे होंगे। 28 साल तक करीब 18 लाख और 30 साल तक करीब 23 से ₹25 लाख। 5 साल की आमदनी करीब 70 से 75 लाख। मतलब 5 साल के अंदर करीब 40 लाख का।


In [9]:
# chunks

In [10]:
len(chunks)

29

## 3. Creating Embeddings and Storing in Vector Database


In [11]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2")

In [12]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
vectorstore = FAISS.from_documents(chunks, embeddings)

In [14]:
vectorstore.index_to_docstore_id

{0: 'cea05c6a-8026-4a9a-a988-da661c1094e6',
 1: '9ac33683-f15e-49d6-8f5a-965776051e55',
 2: '8175363f-1baf-43ba-b29b-191cd88219d5',
 3: '1b124be4-8f3c-48b1-8e3e-d94d1975938c',
 4: 'd1c3461d-6548-432d-ad5c-fafb09e208fe',
 5: 'd6c3e99a-5e7c-4b14-bf35-ccd6bccdc888',
 6: '02592540-1070-425d-93b4-042afa546962',
 7: 'a47fbb11-7600-4021-8e32-163d1e41bd5b',
 8: '0e109f0a-b894-4b67-a7c7-de351a5b5d57',
 9: '0755632d-74c6-48fc-8226-41400e246d00',
 10: '9beeb6e2-54b4-433f-8a91-53ca2d4b8623',
 11: '704817fc-6d34-4bd2-9ab3-4d3edd09e59e',
 12: 'a0fdf0fa-6de6-443a-a402-a480c71cd2c3',
 13: 'ffe43ce6-be94-4b44-99f9-a58f8622a121',
 14: 'f3a68fc4-6ccd-4a11-9256-a915ba5abeb2',
 15: 'f384a30e-d7f2-4e08-a620-69ec82836031',
 16: '5361246d-97ef-455b-92b5-39695ceb01c5',
 17: 'ed16713b-1fbe-45ab-92f6-fc677ca9884d',
 18: '741d2bea-664f-42ce-ac0a-2bb62b52a819',
 19: '108a2273-ff71-4915-9ada-03f0fa5a551d',
 20: '36e5d764-3472-4674-b227-ee413bfad8f9',
 21: 'fc8a0e3f-8fb4-4fc7-97a8-2904321d9c1e',
 22: '7ed15b21-7998-

In [15]:
len(vectorstore.index_to_docstore_id)

29

In [16]:
# vectorstore.get_by_ids(["bcabe210-be6b-45c9-bbe4-b825c40b3159"])[0].page_content

## 4. Creating retriever and Chat Model


In [17]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [18]:
# retriever.invoke("what is atlas?")

In [19]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.3)

In [20]:
prompt = ChatPromptTemplate.from_template(
    """
    You're a helpful assistant. Based on the following context, answer the following question only from provided context.
    if you answer is not found in context, simply say 'answer not found'.

    Context:
    {context}

    Question: {input}

    Answer:
    """
)

## BUILDING FINAL RAG CHAIN WITH RETRIEVER & PROMPT


In [21]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [22]:
document_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, document_chain)

In [23]:
query = "how many programming languages are mentioned in the video? and name those languages."

In [24]:
response = rag_chain.invoke(
    input={"input": query}
)

In [25]:
print("\nRAG Response:")
print(response["answer"])


RAG Response:
Based on the context, one language is mentioned: सीक्वल (Sequel/SQL).
